# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
#findspark.init('C:\\apachespark')
findspark.init('C:\\EPrograms\\spark-2.4.4-bin-hadoop2.7')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Songs  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext.getOrCreate(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"msdchallenge/kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [6]:
triplet_rdd.take(5)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOBONKR12A58A7A7E0', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOEGIYH12A6D4FC0E3', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOFLJQZ12A6D4FADA6', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOHTKMO12AB01843B0', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SODQZCY12A6D4F9D11', '1']]

In [8]:
songs_rdd = sc.textFile(r"msdchallenge/kaggle_songs.txt") \
    .map(lambda line: line.split()) 
songs_rdd.take(5)

[['SOAAADD12AB018A9DD', '1'],
 ['SOAAADE12A6D4F80CC', '2'],
 ['SOAAADF12A8C13DF62', '3'],
 ['SOAAADZ12A8C1334FB', '4'],
 ['SOAAAFI12A6D4F9C66', '5']]

In [36]:
#Create an RDD with msdevalutation_triplets.txt_ and replace the song name with the song index from msdsongs.txt_. 
#Identify the number of songs that do not have any rating.
songs_rdd.map(lambda x : (x[0],(x[1]))) \
         .join(triplet_rdd.map(lambda x: (x[1],(x[0],x[2])))) \
         .map(lambda x : [x[1][1][0],x[1][0],x[1][1][1]]) \
         .take(5)

[['32cf63cf65787ce7e72fc7fda6ee585979af6582', '16', '1'],
 ['d1b88940eabd4fab860edf68f0f0842e4f902c78', '16', '1'],
 ['140be58d1ea6d645ed7b317f61197b97975ac1ba', '29', '1'],
 ['bb2e92130a312d9f9e5754f05b36ca85505656b0', '42', '1'],
 ['12e71abacd59cd98b403ffc0444b19dc3ba0b0d6', '48', '1']]

In [32]:
triplet_rdd.filter(lambda x: x[0] == '32cf63cf65787ce7e72fc7fda6ee585979af6582' ).collect()

[['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOKONZI12A6D4FA17B', '1'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOIINQC12A6D4F73A6', '1'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOAAAQN12AB01856D3', '1'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SODOQUU12AAF3B33DF', '1'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOFGDGA12AB017C86B', '1'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOKVKON12A6D4F6DC8', '1'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOYJNHO12AB01856DC', '2'],
 ['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOOBBJV12A8151CE9B', '1']]

In [35]:
triplet_rdd.filter(lambda x: x[1] == 'SOAAAQN12AB01856D3' ).collect()

[['32cf63cf65787ce7e72fc7fda6ee585979af6582', 'SOAAAQN12AB01856D3', '1'],
 ['d1b88940eabd4fab860edf68f0f0842e4f902c78', 'SOAAAQN12AB01856D3', '1']]

In [34]:
songs_rdd.filter(lambda x: x[1] == '16').collect()

[['SOAAAQN12AB01856D3', '16']]

In [37]:
songs_rdd.map(lambda x : (x[0],(x[1]))) \
            .leftOuterJoin(triplet_rdd.map(lambda x: (x[1],(x[0],x[2])))) \
            .filter(lambda x : x[1][1] == None).collect()

[('SOAAALJ12AB01828B4', ('12', None)),
 ('SOAABDX12A58A7E508', ('25', None)),
 ('SOAABRF12AC468803F', ('34', None)),
 ('SOAABYG12AB01876F4', ('40', None)),
 ('SOAACGC12A8C1311C6', ('47', None)),
 ('SOAACNA12AB01867BA', ('51', None)),
 ('SOAACNQ12A8AE4692D', ('53', None)),
 ('SOAACTC12AB0186A20', ('60', None)),
 ('SOAACXA12AB018A864', ('64', None)),
 ('SOAACZY12A6D4F8081', ('67', None)),
 ('SOAADAG12AB018997B', ('69', None)),
 ('SOAADAS12A58A784EC', ('70', None)),
 ('SOAADJU12AB01872C7', ('78', None)),
 ('SOAADVA12A8C130133', ('89', None)),
 ('SOAADWH12A58A7A651', ('91', None)),
 ('SOAAETS12AB0180FAE', ('117', None)),
 ('SOAAEUY12AB017FBA0', ('120', None)),
 ('SOAAEXS12A58A784B3', ('121', None)),
 ('SOAAFBJ12AB018E272', ('125', None)),
 ('SOAAFDI12AB018888E', ('128', None)),
 ('SOAAFEW12A8AE47AE5', ('131', None)),
 ('SOAAFVG12A8C13C107', ('146', None)),
 ('SOAAGBA12AB018126F', ('153', None)),
 ('SOAAGQU12AB01843A8', ('169', None)),
 ('SOAAHMO12A8C13EC21', ('186', None)),
 ('SOAAHOU12A58

In [38]:
triplet_rdd.filter(lambda x: x[1] == 'SOAAALJ12AB01828B4' ).collect()

[]

## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 